# Caten: Python Bindings for ISL - Tutorial


Caten provides Python bindings for the Integer Set Library (ISL). It features automatic memory management, comprehensive type hints, and a Pythonic API.


## 1. Setup and Implicit Context


You can start using ISL objects immediately. The library context is managed automatically, so you don't need to explicitly allocate or free it.


In [3]:
import sys
sys.path.append("../")
import caten.isl as I

# You can verify the context is working by creating an object
s = I.Set("{ [i] : 0 <= i < 10 }")
print(f"Set created: {s}")

Set created: { [i] : 0 <= i <= 9 }


## 2. Sets and Basic Operations


Sets (`isl_set`) are the core data structure in ISL. They represent sets of integer tuples defined by affine constraints.


In [4]:
# Create sets from string representations
s1 = I.Set("{ [i] : 0 <= i < 10 }")
s2 = I.Set("{ [i] : 5 <= i < 15 }")

print(f"s1: {s1}")
print(f"s2: {s2}")

# Intersection
s_inter = s1.intersect(s2)
print(f"Intersection (s1 ∩ s2): {s_inter}")

# Union
s_union = s1.union(s2)
print(f"Union (s1 ∪ s2): {s_union}")

# Difference
s_diff = s1.subtract(s2)
print(f"Difference (s1 - s2): {s_diff}")

# Check properties
print(f"Is intersection empty? {s_inter.is_empty()}")
print(f"Is s1 a subset of s_union? {s1.is_subset(s_union)}")

s1: { [i] : 0 <= i <= 9 }
s2: { [i] : 5 <= i <= 14 }
Intersection (s1 ∩ s2): { [i] : 5 <= i <= 9 }
Union (s1 ∪ s2): { [i] : 0 <= i <= 14 and (i <= 9 or i >= 5) }
Difference (s1 - s2): { [i] : 0 <= i <= 4 }
Is intersection empty? False
Is s1 a subset of s_union? True


## 3. Maps (Relations)


Maps (`isl_map`) represent relations between integer tuples. They map a domain set to a range set.


In [5]:
# Create a map
m = I.Map("{ [i] -> [i+1] : 0 <= i < 10 }")
print(f"Map: {m}")

# Apply map to a set
# Image of s1 under m: { y | exists x in s1 s.t. (x, y) in m }
s_image = s1.apply(m)
print(f"Image of s1 under m: {s_image}")

# Intersection of domain
m_restricted = m.intersect_domain(I.Set("{ [i] : i < 5 }"))
print(f"Restricted Map: {m_restricted}")

Map: { [i] -> [1 + i] : 0 <= i <= 9 }
Image of s1 under m: { [i0] : 0 < i0 <= 10 }
Restricted Map: { [i] -> [1 + i] : 0 <= i <= 4 }


## 4. Spaces and Identifiers


Spaces (`isl_space`) define the dimensionality and structure of sets and maps.


In [6]:
# Create a set space with 0 parameters and 2 dimensions
space = I.Space.set_alloc(0, 2) 

# Set tuple name
space = space.set_tuple_name(I.ISLDimType.ISL_DIM_SET, "Point")
print(f"Space: {space}")

# Create an Identifier
id_a = I.Id("A")
print(f"Identifier: {id_a}")

Space: { Point[i0, i1] }
Identifier: A


## 5. Affine Expressions


Affine expressions (`isl_aff`, `isl_multi_aff`, `isl_pw_aff`) represent linear functions.


In [7]:
# Simple affine expression
aff = I.Aff("{ [i] -> [2*i + 1] }")
print(f"Aff: {aff}")

# Piecewise affine expression (different expressions for different domains)
pw_aff = I.PwAff("{ [i] -> [i] : i >= 0; [i] -> [-i] : i < 0 }")
print(f"PwAff (abs): {pw_aff}")

# Multi-affine expression (tuple of affine expressions)
ma = I.MultiAff("{ [i,j] -> [i+j, i-j] }")
print(f"MultiAff: {ma}")

Aff: { [i] -> [(1 + 2i)] }
PwAff (abs): { [i] -> [(i)] : i >= 0; [i] -> [(-i)] : i < 0 }
MultiAff: { [i, j] -> [(i + j), (i - j)] }


## 6. Union Types


Union sets (`isl_union_set`) and union maps (`isl_union_map`) can contain relations over different spaces.


In [8]:
us = I.UnionSet("{ A[i] : 0 <= i < 10; B[j] : 0 <= j < 20 }")
print(f"UnionSet: {us}")

# Operations work similarly to basic sets
us2 = I.UnionSet("{ A[i] : i > 5 }")
print(f"Union Intersection: {us.intersect(us2)}")

UnionSet: { A[i] : 0 <= i <= 9; B[j] : 0 <= j <= 19 }
Union Intersection: { A[i] : 6 <= i <= 9 }


## 7. Values


ISL handles arbitrary precision integers using `isl_val`.


In [9]:
v = I.Val.int_from_si(42)
print(f"Value: {v}")
print(f"Is positive? {v.sgn() > 0}")

# Operations
v2 = v.add(I.Val.int_from_si(10))
print(f"v + 10 = {v2}")

Value: 42
Is positive? True
v + 10 = 52


## 8. Mixin Features: Pythonic Operators


Caten objects inherit from `ISLObjectMixin`, which provides operator overloading for intuitive usage. You can use standard Python operators (`|`, `&`, `-`, `+`, `*`, etc.) to perform ISL operations.



| Operator | ISL Function | Description |
|---|---|---|
| `a | b` | `union` | Set union |
| `a & b` | `intersect` | Set intersection |
| `a - b` | `subtract` | Set difference |
| `a + b` | `add` / `sum` | Addition |
| `a <= b` | `is_subset` | Subset check |
| `a == b` | `is_equal` | Equality check |


In [10]:
s1 = I.Set("{ [i] : 0 <= i < 10 }")
s2 = I.Set("{ [i] : 5 <= i < 15 }")

# Union (|)
print(f"s1 | s2: {s1 | s2}")

# Intersection (&)
print(f"s1 & s2: {s1 & s2}")

# Difference (-)
print(f"s1 - s2: {s1 - s2}")

# Subset check (<=)
subset = I.Set("{ [i] : 2 <= i <= 8 }")
print(f"subset <= s1: {subset <= s1}")

# Equality check (==)
print(f"s1 == s1: {s1 == s1}")

s1 | s2: { [i] : 0 <= i <= 14 and (i <= 9 or i >= 5) }
s1 & s2: { [i] : 5 <= i <= 9 }
s1 - s2: { [i] : 0 <= i <= 4 }
subset <= s1: True
s1 == s1: True
